# Implementation of A* Algorithm for 8-Puzzle Problem

**Programming Assignment-1**

**Course: Artificial Intelligence (CIS-579)**

**Author: Nithesh Veerappa**

**Student ID: 0188 3074**

**Date: 10/12/2022**

An A* Algorithm is developed to solve the 8-puzzle problem considering the windy situation using modified manhattan distance as the heuristic function h(n) and accumulated cost incurred for every move taken as g(n) with respect to the windy situation specified in the problem statement. The total cost is formulated as the sum of h(n) and g(n) and this is used as the evaluation function to maneuver the agent through the 8-puzzle grid to achieve the goal state, optimally.

## Step 1 : Class Definition

The first step is to create a class that defines all the fucntions necessary to facilitate the program to evaluate the cost (*costgridcal*) and update its grid/state based on the cost estimation (*updatecurrgrid*). The code defining the class is attached below.

In [1]:
import numpy as np

#Class creates a library of fucntion that facilitate the algorithm to make its move to achieve goal state.
class AStarHelp:
    def __init__(self):
        pass
    
    # Heuristic function (h(n))that estimates the cost of the current state against the goal state.
    # Inputs: cost_grid (zero Initialization on cost grid), temp_grid (Grid after taking a temp move), goal_grid (goal state)
    # Outputs: cost_grid (Cost grid after taking a temp move)
    def costgridcal(self,cost_grid, temp_grid, goal_grid):
        for i in np.arange(1,9):
            if i > 0:
                i1,i2 = np.where(temp_grid == i)
                g1,g2 = np.where(goal_grid == i)
                
                if i1[0] > g1[0]:
                    x_cost = abs(i1[0] - g1[0])*1 
                elif i1[0] < g1[0]:
                    x_cost = abs(i1[0]-g1[0])*3
                else:
                    x_cost = 0
                    
                if i2[0] > g2[0]:
                    y_cost = abs(i2[0] - g2[0])*2
                elif i2[0] < g2[0]:
                    y_cost = abs(i2[0] - g2[0])*2
                else:
                    y_cost = 0
                    
                cost_grid[i1[0]][i2[0]] = x_cost + y_cost
        return cost_grid
    
    # Function updates the current state by taking any one of the actions of moving to W,N,E or S based on the evaluation cost.
    # Inputs: min_idx (index of the minimum cost in priority queue), curr_grid (last updated state), cc(last accumulated cost)
    # Outputs: curr_grid (upated grid afte making any one of the four moves move), cc(upadted acculmulated cost)
    def updatecurrgrid(self,min_idx,curr_grid,cc):
        start_node = np.where(curr_grid == 0)
        x = start_node[0][0]
        y = start_node[1][0]
        if min_idx == 0:
            try:
                curr_grid[x][y-1],curr_grid[x][y] = curr_grid[x][y], curr_grid[x][y-1] #Swapping the numbers for West move
                cc = cc + 2 #Adding cost due to west move to the previous accumulation
            except:
                pass
                    
        elif min_idx == 1:
            try:
                curr_grid[x-1][y],curr_grid[x][y] = curr_grid[x][y],curr_grid[x-1][y] #Swapping the numbers for North move
                cc = cc + 3 #Adding cost due to south move to the previous accumulation
            except:
                pass
            
        elif min_idx == 2:
            try:
                curr_grid[x][y+1],curr_grid[x][y] = curr_grid[x][y],curr_grid[x][y+1] #Swapping the numbers for East Move
                cc = cc + 2 #Adding cost due to east move to the previous accumulation
            except:
                pass
    
            
        elif min_idx == 3:
            try:
                curr_grid[x+1][y],curr_grid[x][y] = curr_grid[x][y],curr_grid[x+1][y] #Swapping the numbers for South Move
                cc = cc + 1 #Adding cost due to North move to the previous accumulation
            except:
                pass   
        return curr_grid,cc

## Step 2: A-Star Algorithm Logic Definition

This part of the code implements the bulk of the A* algorthim logic. It begins with estimating the evaluation cost (h(n) + g(n)) for making a move in every direction possible and sets up a priority queue. Additionaly, a history is logged for all the explored states/grid configuration to simply bypass them next time when the same grid is encountered during the exploration, therby pruning the search tree saving the computational cost on the entire program.

Once the priority queue and history of the explored states are logged, the program makes a move that is inexpensive in its evaluvated cost and updates the current state to repeat the process again until the goal state is achieved. Sometimes there may exist a multiple feasible moves with same evaluation cost and in such cases the the program assumes the first in first out (FIFO) theory to go ahead with the update, that is to say the first potential move as listed in the priority queue is assumed

The code that implements the A* algorithm is attached below and result of the search is printed.

In [2]:
# Importing all the necessary libraries
import numpy as np

In [3]:
# Initialization of the variables
curr_grid = np.array([[2,8,3],[6,7,4],[1,5,0]]) #Start state. Zero is regarded as blank state or the agent that moves around.
goal_grid = np.array([[1,2,3],[8,0,4],[7,6,5]]) #Goal state. Zero is regarded as blank state or the agent that moves around.
cc = 0 # Zero accumulation of the cost incurred after making a move
count = 0 # step count in the search
start_node = np.where(curr_grid == 0) # Searcing for the coordinates of the blank state/agent in start grid
x,y = start_node[0][0], start_node[1][0] # coordinates of the blank state/agent in start grid
flt = np.append(curr_grid.flatten(),[x,y])
exp_st = [hash(np.array_str(flt))] # Hash function that generates a key to the start state and adds to the history
cost_grid = np.array(np.zeros((3,3)))
currgridcostcal = AStarHelp()
init_cost = sum(sum(currgridcostcal.costgridcal(cost_grid,curr_grid, goal_grid)))

In [4]:
# A* Algorithm Logic
while np.allclose(curr_grid, goal_grid) != True:  #To repeat the search process until goal state is achieved.
    priority_q = [] # Priority queue reset and initialization
    temp_grid = np.copy(curr_grid)  #creating a copy of the current state to not modify the original current grid

    #Estimation of the evaluation cost and logging the priority queue and state exploration after making a move to the west
    if (y-1) >= 0:  
        temp_grid[x][y-1],temp_grid[x][y] = temp_grid[x][y],temp_grid[x][y-1] #West move
        flt1 = temp_grid.flatten()
        hash1 = hash(np.array_str(flt1))
        if (hash1 in exp_st) != True:
            cost_grid = np.array(np.zeros((3,3)))
            costgridcal = AStarHelp()
            new_cg = sum(sum(costgridcal.costgridcal(cost_grid,temp_grid, goal_grid)))
            priority_q = np.append(priority_q, new_cg + cc) # updating the priority queue
            exp_st = np.hstack((exp_st,hash1)) # updating the history of explored states
        else:
            priority_q = np.append(priority_q, 1e30) # when the state is already explored, high cost is privided to skip
    else:
        priority_q = np.append(priority_q, 1e30) # when the agent is beside the grid wall, high cost is privided to skip
    temp_grid = np.copy(curr_grid)
    
    #Estimation of the evaluation cost and logging the priority queue and state exploration after making a move to the North
    if (x-1) >= 0:
        temp_grid[x-1][y],temp_grid[x][y] = temp_grid[x][y],temp_grid[x-1][y] #Nortth move
        flt2 = temp_grid.flatten()
        hash2 = hash(np.array_str(flt2))
        if (hash2 in exp_st) != True:
            cost_grid = np.array(np.zeros((3,3)))
            costgridcal = AStarHelp()
            new_cg = sum(sum(costgridcal.costgridcal(cost_grid,temp_grid, goal_grid)))
            priority_q = np.append(priority_q, new_cg + cc) # updating the priority queue
            exp_st = np.hstack((exp_st,hash2)) # updating the history of explored states
        else:
            priority_q = np.append(priority_q, 1e30) # when the state is already explored, high cost is privided to skip
    else:
        priority_q = np.append(priority_q, 1e30) # when the agent is beside the grid wall, high cost is privided to skip
    temp_grid = np.copy(curr_grid)
    
    #Estimation of the evaluation cost and logging the priority queue and state exploration after making a move to the East
    if (y+1) <= 2: 
        temp_grid[x][y+1],temp_grid[x][y] = temp_grid[x][y],temp_grid[x][y+1] #East Move
        flt3 = temp_grid.flatten()
        hash3 = hash(np.array_str(flt3))
        if (hash3 in exp_st) != True:
            cost_grid = np.array(np.zeros((3,3)))
            costgridcal = AStarHelp()
            new_cg = sum(sum(costgridcal.costgridcal(cost_grid,temp_grid, goal_grid)))
            priority_q = np.append(priority_q, new_cg + cc) # updating the priority queue
            exp_st = np.hstack((exp_st,hash3)) # updating the history of explored states
        else:
            priority_q = np.append(priority_q, 1e30) # when the state is already explored, high cost is privided to skip
    else:
        priority_q = np.append(priority_q, 1e30) # when the agent is beside the grid wall, high cost is privided to skip
    temp_grid = np.copy(curr_grid)

    #Estimation of the evaluation cost and logging the priority queue and state exploration after making a move to the South
    if (x+1) <= 2:
        temp_grid[x+1][y],temp_grid[x][y] = temp_grid[x][y],temp_grid[x+1][y] #South move
        flt4 = temp_grid.flatten()
        hash4 = hash(np.array_str(flt4))
        if (hash4 in exp_st) != True:
            cost_grid = np.array(np.zeros((3,3)))
            costgridcal = AStarHelp()
            new_cg = sum(sum(costgridcal.costgridcal(cost_grid,temp_grid, goal_grid)))
            priority_q = np.append(priority_q, new_cg + cc) # updating the priority queue
            exp_st = np.hstack((exp_st,hash4)) # updating the history of explored states
        else:
            priority_q = np.append(priority_q, 1e30) # when the state is already explored, high cost is privided to skip
    else:
        priority_q = np.append(priority_q, 1e30) # when the agent is beside the grid wall, high cost is privided to skip
    temp_grid = np.copy(curr_grid)
    
    # Obtaining the move with least cost from the priority queue
    min_idx = np.where(priority_q <= init_cost)
    min_idx = (min_idx[0]).tolist()
    gridupdate = AStarHelp() #creating an object to call the function updatecurrgrid
    
    # Updating the grid for the move with minimal cost
    if len(min_idx) < 2:
        min_idx = min_idx[0]
        curr_grid, cc = gridupdate.updatecurrgrid(min_idx, curr_grid,cc)
    else:
        min_idx = min_idx[0]
        curr_grid, cc = gridupdate.updatecurrgrid(min_idx, curr_grid,cc)
        
    count = count + 1  #Incrementing the step count
    start_node = np.where(curr_grid == 0) #Finding the cordinates of the agent after grid update
    x,y = start_node[0][0], start_node[1][0]
    cost_grid = np.array(np.zeros((3,3))) # Resetting the cost grid
    
    # Printing the output of the search after every move
    print('Step number: {}'. format(count))
    print(curr_grid)
    print('Accumulated cost = {}'.format(cc))
    print('Heuristic cost = {}'.format(sum(sum(costgridcal.costgridcal(cost_grid, curr_grid, goal_grid)))))
    print('---------------------------------------')
    
if np.allclose(curr_grid, goal_grid) == True:
    print('Goal state is reached and therefore search is terminated')

Step number: 1
[[2 8 3]
 [6 7 4]
 [1 0 5]]
Accumulated cost = 2
Heuristic cost = 19.0
---------------------------------------
Step number: 2
[[2 8 3]
 [6 0 4]
 [1 7 5]]
Accumulated cost = 5
Heuristic cost = 16.0
---------------------------------------
Step number: 3
[[2 8 3]
 [0 6 4]
 [1 7 5]]
Accumulated cost = 7
Heuristic cost = 14.0
---------------------------------------
Step number: 4
[[2 8 3]
 [1 6 4]
 [0 7 5]]
Accumulated cost = 8
Heuristic cost = 13.0
---------------------------------------
Step number: 5
[[2 8 3]
 [1 6 4]
 [7 0 5]]
Accumulated cost = 10
Heuristic cost = 11.0
---------------------------------------
Step number: 6
[[2 8 3]
 [1 0 4]
 [7 6 5]]
Accumulated cost = 13
Heuristic cost = 8.0
---------------------------------------
Step number: 7
[[2 0 3]
 [1 8 4]
 [7 6 5]]
Accumulated cost = 16
Heuristic cost = 5.0
---------------------------------------
Step number: 8
[[0 2 3]
 [1 8 4]
 [7 6 5]]
Accumulated cost = 18
Heuristic cost = 3.0
-------------------------------